In [1]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")


In [2]:
projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
pprint.pprint(datasetDict)
pprint.pprint(outputDict)

D:\project_meta\NNproject\NNI
{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
{'ECG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'ECG_uniformfilted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'EEG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\EEG',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'EEG_unifor

# 模型

In [3]:
from project.model.CNN1D import *
from torchvision.datasets import *
from torchvision.transforms import *
import torchvision
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
MNISTDataset=FashionMNIST(root=r'D:/dataset',transform=train_transforms)


In [4]:
from typing import Any, Optional
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT
from torchmetrics.functional import accuracy
class Base(pl.LightningModule):
    def __init__(self, net,classes,lossFuncthion,optimizer,lrScheduler,*args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)  
        self.Net=net
        self.LossFunction=nn.CrossEntropyLoss()
        self.Optimizer=optimizer
        self.classes=classes
        
        # self.LrScheduler=lrScheduler

    def forward(self,x, *args: Any, **kwargs: Any) -> Any:
        y=self.Net(x)
        return y#torch.rand([1,10])
    def configure_optimizers(self) -> Any:
        return [self.Optimizer]
    def training_step(self, batch,*args: Any, **kwargs: Any) -> STEP_OUTPUT:
        x,y=batch
        y_hat=self.forward(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=self.classes)
        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("trainloss", loss, on_step=True, on_epoch=True)
        return {'loss':loss}
    def validation_step(self,batch, *args: Any, **kwargs: Any) -> STEP_OUTPUT | None:
        x,y=batch
        y_hat=self.Net(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=self.classes)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        self.log("testloss", loss, on_step=False, on_epoch=True)
        return {'loss':loss}
        

In [6]:
from torchsummary import summary
from project.model.MLP import *
from torchvision.models import resnet50,ResNet50_Weights
net=resnet50(ResNet50_Weights.DEFAULT)
net.conv1=torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
net.fc= torch.nn.Linear(in_features=2048, out_features=10, bias=True)
Base(net,10,nn.CrossEntropyLoss(),torch.optim.Adam(net.parameters(),lr=0.01),lrScheduler=None)


g:\install\anaconda\envs\nni\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Base(
  (Net): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Co

In [7]:
plnet=Base(net,lossFuncthion='none',classes=10,optimizer=torch.optim.Adam(net.parameters(),lr=0.01),lrScheduler=None)
summary(plnet,(1,28,28),batch_size=2,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [2, 64, 14, 14]           3,136
       BatchNorm2d-2            [2, 64, 14, 14]             128
              ReLU-3            [2, 64, 14, 14]               0
         MaxPool2d-4              [2, 64, 7, 7]               0
            Conv2d-5              [2, 64, 7, 7]           4,096
       BatchNorm2d-6              [2, 64, 7, 7]             128
              ReLU-7              [2, 64, 7, 7]               0
            Conv2d-8              [2, 64, 7, 7]          36,864
       BatchNorm2d-9              [2, 64, 7, 7]             128
             ReLU-10              [2, 64, 7, 7]               0
           Conv2d-11             [2, 256, 7, 7]          16,384
      BatchNorm2d-12             [2, 256, 7, 7]             512
           Conv2d-13             [2, 256, 7, 7]          16,384
      BatchNorm2d-14             [2, 25

In [27]:
from torch.utils.data import random_split
train, val = random_split(MNISTDataset, [55000, 5000])
trainer = pl.Trainer()
trainer.progress_bar_refresh_rate = 10
trainer.fit(plnet, DataLoader(train,batch_size=64),DataLoader(val,batch_size=64))#,

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | Net          | ResNet           | 23.5 M
1 | LossFunction | CrossEntropyLoss | 0     
--------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.089    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]